In [137]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import functions as fun
from linearmodels.iv import IV2SLS

pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50

import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

In [138]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,1.037e-02,Aiways,U5,400,2.846e+05,201,34,SUV,C,CN,257,1.0
9,1,2022,5.976e-03,Aiways,U5,400,3.137e+05,201,34,SUV,C,CN,183,1.0
10,1,2023,2.860e-03,Aiways,U5,400,2.645e+05,201,34,SUV,C,CN,177,1.0
21,2,2023,4.848e-05,Aiways,U6,405,3.606e+05,214,34,SUV,C,CN,3,1.0
28,3,2019,4.063e-02,Audi,e-tron,375,9.797e+05,402,17,SUV,F,DE,222,1.0
29,3,2020,3.468e-02,Audi,e-tron,375,8.901e+05,402,17,SUV,F,DE,491,1.0
30,3,2021,1.049e-02,Audi,e-tron,375,8.000e+05,402,17,SUV,F,DE,260,1.0
31,3,2022,1.757e-02,Audi,e-tron,375,7.897e+05,402,17,SUV,F,DE,538,1.0
32,3,2023,1.099e-03,Audi,e-tron,375,6.730e+05,402,17,SUV,F,DE,68,1.0
41,4,2021,3.391e-03,Audi,e-tron GT,472,1.279e+06,522,17,Sedan,F,DE,84,1.0


In [139]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,1.037e-02,Aiways,U5,400,2.846e+05,201,34,SUV,C,CN,257,1.0
1,1,2022,5.976e-03,Aiways,U5,400,3.137e+05,201,34,SUV,C,CN,183,1.0
2,1,2023,2.860e-03,Aiways,U5,400,2.645e+05,201,34,SUV,C,CN,177,1.0
3,2,2023,4.848e-05,Aiways,U6,405,3.606e+05,214,34,SUV,C,CN,3,1.0
4,3,2019,4.063e-02,Audi,e-tron,375,9.797e+05,402,17,SUV,F,DE,222,1.0
5,3,2020,3.468e-02,Audi,e-tron,375,8.901e+05,402,17,SUV,F,DE,491,1.0
6,3,2021,1.049e-02,Audi,e-tron,375,8.000e+05,402,17,SUV,F,DE,260,1.0
7,3,2022,1.757e-02,Audi,e-tron,375,7.897e+05,402,17,SUV,F,DE,538,1.0
8,3,2023,1.099e-03,Audi,e-tron,375,6.730e+05,402,17,SUV,F,DE,68,1.0
9,4,2021,3.391e-03,Audi,e-tron GT,472,1.279e+06,522,17,Sedan,F,DE,84,1.0


In [140]:
fun.GH(data, 'Range', 0.5)
fun.GH(data, 'HP', 0.5)
fun.GH(data, 'Chargetime', 0.5)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,Range_GH,HP_GH,Chargetime_GH
0,1,2021,1.037e-02,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0,14999,7405,1317
1,1,2022,5.976e-03,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0,21535,10602,1796
2,1,2023,2.860e-03,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0,27923,15010,2099
3,2,2023,4.848e-05,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0,27468,14562,2099
4,3,2019,4.063e-02,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0,4709,3105,533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,189,2023,6.464e-05,Volvo,EX30,475,368245.000,268,28,SUV,B,SE,4,1.0,21894,17776,1813
330,190,2021,1.465e-02,Volvo,XC40,457,462060.600,402,28,SUV,C,SE,363,1.0,11699,10265,1138
331,190,2022,3.331e-02,Volvo,XC40,457,416263.400,402,28,SUV,C,SE,1020,1.0,17028,16217,1483
332,190,2023,3.175e-02,Volvo,XC40,457,439266.600,402,28,SUV,C,SE,1965,1.0,22067,19786,1813


In [141]:
product_data = data.rename(columns={
    'Year': 'market_ids',
    'Model': 'product_ids',
    'Market_share': 'shares',
    'Price': 'prices',
    'Manufacturer': 'firm_ids',
    'Range_GH': 'demand_instruments0',
    'HP_GH': 'demand_instruments1',
    'Chargetime_GH': 'demand_instruments2',
})

In [142]:
#Scale for better intepretation
product_data['prices'] = product_data['prices']/10_000 #(Change in ms(%) for change in pris in 10.000)
product_data['HP'] = product_data['HP']/10           #(Change in ms(%) for change in HP in 10)
product_data['Range'] = product_data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [143]:
# Creating dummy for china
product_data['China'] = (product_data['Country'] == 'CN').astype(int)

# Outside share

In [144]:
product_data.loc[product_data['market_ids'] == 2013, 'shares'] = product_data.loc[product_data['market_ids'] == 2013, 'Sales'] / 180632
product_data.loc[product_data['market_ids'] == 2014, 'shares'] = product_data.loc[product_data['market_ids'] == 2014, 'Sales'] / 188406
product_data.loc[product_data['market_ids'] == 2015, 'shares'] = product_data.loc[product_data['market_ids'] == 2015, 'Sales'] / 206653
product_data.loc[product_data['market_ids'] == 2016, 'shares'] = product_data.loc[product_data['market_ids'] == 2016, 'Sales'] / 222471
product_data.loc[product_data['market_ids'] == 2017, 'shares'] = product_data.loc[product_data['market_ids'] == 2017, 'Sales'] / 221471
product_data.loc[product_data['market_ids'] == 2018, 'shares'] = product_data.loc[product_data['market_ids'] == 2018, 'Sales'] / 252328
product_data.loc[product_data['market_ids'] == 2019, 'shares'] = product_data.loc[product_data['market_ids'] == 2019, 'Sales'] / 258727
product_data.loc[product_data['market_ids'] == 2020, 'shares'] = product_data.loc[product_data['market_ids'] == 2020, 'Sales'] / 230060
product_data.loc[product_data['market_ids'] == 2021, 'shares'] = product_data.loc[product_data['market_ids'] == 2021, 'Sales'] / 222210
product_data.loc[product_data['market_ids'] == 2022, 'shares'] = product_data.loc[product_data['market_ids'] == 2022, 'Sales'] / 181030
product_data.loc[product_data['market_ids'] == 2023, 'shares'] = product_data.loc[product_data['market_ids'] == 2023, 'Sales'] / 203690

In [145]:
product_data['outside_share'] = 1 - product_data.groupby('market_ids')['shares'].transform('sum')
product_data[['shares', 'outside_share']].describe()

,shares,outside_share
count,3.340e+02,334.000
mean,2.128e-03,0.841
std,5.618e-03,0.109
min,4.347e-06,0.696
25%,1.227e-04,0.696
50%,7.149e-04,0.831
75%,2.161e-03,0.938
max,8.814e-02,0.998


# Demand

In [146]:
product_data['logit_delta'] = np.log(product_data['shares'] / product_data['outside_share'])
formula = 'logit_delta ~ 1 + [prices ~ demand_instruments0 + demand_instruments1 + demand_instruments2] + Range + HP + Chargetime + China'
IV = IV2SLS.from_formula(formula, product_data).fit(cov_type='robust')
IV.first_stage

,prices
R-squared,0.6105
Partial R-squared,0.0919
Shea's R-squared,0.0919
Partial F-statistic,22.829
P-value (Partial F-stat),4.383e-05
Partial F-stat Distn,chi2(3)
==========================,===========
Intercept,1.7422
,(0.2392)
Range,0.4165


In [147]:
IV.summary#.tables[1]

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:            logit_delta   R-squared:                      0.0795
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0654
No. Observations:                 334   F-statistic:                    145.01
Date:                Mon, Jun 03 2024   P-value (F-stat)                0.0000
Time:                        23:33:22   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -8.0044     0.7315    -10.942     0.0000     -9.4382     -6.5707
Range          0.1312     0.0148     8.8514     0.0000      0.1021      0.1603
HP             0.0247     0.0255     0.9706     0.3317     -0.0252      0.0747
Chargetime    -0.0519     0.0113    -4.5869     0.0000     -0.0740     -0.0297
China         -1.4436     0.3839    -3.7599     0.0002     -2.1961     -0.6911
prices        -0.0712     0.0202    -3.5224     0.0004     -0.1108     -0.0316
==============================================================================

Endogenous: prices
Instruments: demand_instruments0, demand_instruments1, demand_instruments2
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Cost

In [148]:
GH_problem = pyblp.Problem(pyblp.Formulation('1 + prices + Range + HP + Chargetime + China'), product_data)
GH_results = GH_problem.solve(method='1s')

In [149]:
data2023 = product_data[product_data['market_ids'] == 2023]
ols_elasticities = GH_results.compute_elasticities(market_id=2023) # name = 'Range' for elasticities of Range
pd.DataFrame(ols_elasticities, index = data2023['product_ids'], columns=data2023['product_ids'])

product_ids,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,iX,iX1,iX3,Atto 3,Dolphin,Han,Seal,Tang,Berlingo,C4,ë-C4 X,e-SpaceTourer,Jumpy,Born,Spring,...,Taycan,Megane,Zoe,Enyaq iV,Fortwo,Korando,Solterra,Model 3,Model S,Model X,Model Y,bZ4X,Proace City Verso,Proace Verso,Golf,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
product_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
U5,-1.881,3.781e-05,0.002,9.800e-04,0.054,0.013,1.107e-05,0.034,0.009,0.003,0.014,0.017,0.016,0.002,0.002,6.189e-05,3.585e-04,9.787e-05,2.970e-04,0.002,0.002,6.468e-04,1.000e-04,0.015,5.001e-04,...,0.007,0.014,2.663e-04,0.046,2.206e-05,3.988e-05,6.606e-04,0.053,0.004,0.003,0.254,0.014,1.031e-04,2.905e-04,1.092e-05,0.014,0.037,0.014,0.003,0.022,4.827e-04,0.008,5.147e-05,0.030,1.411e-04
U6,0.002,-2.567e+00,0.002,9.800e-04,0.054,0.013,1.107e-05,0.034,0.009,0.003,0.014,0.017,0.016,0.002,0.002,6.189e-05,3.585e-04,9.787e-05,2.970e-04,0.002,0.002,6.468e-04,1.000e-04,0.015,5.001e-04,...,0.007,0.014,2.663e-04,0.046,2.206e-05,3.988e-05,6.606e-04,0.053,0.004,0.003,0.254,0.014,1.031e-04,2.905e-04,1.092e-05,0.014,0.037,0.014,0.003,0.022,4.827e-04,0.008,5.147e-05,0.030,1.411e-04
e-tron,0.002,3.781e-05,-4.789,9.800e-04,0.054,0.013,1.107e-05,0.034,0.009,0.003,0.014,0.017,0.016,0.002,0.002,6.189e-05,3.585e-04,9.787e-05,2.970e-04,0.002,0.002,6.468e-04,1.000e-04,0.015,5.001e-04,...,0.007,0.014,2.663e-04,0.046,2.206e-05,3.988e-05,6.606e-04,0.053,0.004,0.003,0.254,0.014,1.031e-04,2.905e-04,1.092e-05,0.014,0.037,0.014,0.003,0.022,4.827e-04,0.008,5.147e-05,0.030,1.411e-04
e-tron GT,0.002,3.781e-05,0.002,-8.678e+00,0.054,0.013,1.107e-05,0.034,0.009,0.003,0.014,0.017,0.016,0.002,0.002,6.189e-05,3.585e-04,9.787e-05,2.970e-04,0.002,0.002,6.468e-04,1.000e-04,0.015,5.001e-04,...,0.007,0.014,2.663e-04,0.046,2.206e-05,3.988e-05,6.606e-04,0.053,0.004,0.003,0.254,0.014,1.031e-04,2.905e-04,1.092e-05,0.014,0.037,0.014,0.003,0.022,4.827e-04,0.008,5.147e-05,0.030,1.411e-04
Q4 e-tron,0.002,3.781e-05,0.002,9.800e-04,-4.634,0.013,1.107e-05,0.034,0.009,0.003,0.014,0.017,0.016,0.002,0.002,6.189e-05,3.585e-04,9.787e-05,2.970e-04,0.002,0.002,6.468e-04,1.000e-04,0.015,5.001e-04,...,0.007,0.014,2.663e-04,0.046,2.206e-05,3.988e-05,6.606e-04,0.053,0.004,0.003,0.254,0.014,1.031e-04,2.905e-04,1.092e-05,0.014,0.037,0.014,0.003,0.022,4.827e-04,0.008,5.147e-05,0.030,1.411e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,0.002,3.781e-05,0.002,9.800e-04,0.054,0.013,1.107e-05,0.034,0.009,0.003,0.014,0.017,0.016,0.002,0.002,6.189e-05,3.585e-04,9.787e-05,2.970e-04,0.002,0.002,6.468e-04,1.000e-04,0.015,5.001e-04,...,0.007,0.014,2.663e-04,0.046,2.206e-05,3.988e-05,6.606e-04,0.053,0.004,0.003,0.254,0.014,1.031e-04,2.905e-04,1.092e-05,0.014,0.037,0.014,0.003,0.022,-1.244e+00,0.008,5.147e-05,0.030,1.411e-04
C40,0.002,3.781e-05,0.002,9.800e-04,0.054,0.013,1.107e-05,0.034,0.009,0.003,0.014,0.017,0.016,0.002,0.002,6.189e-05,3.585e-04,9.787e-05,2.970e-04,0.002,0.002,6.468e-04,1.000e-04,0.015,5.001e-04,...,0.007,0.014,2.663e-04,0.046,2.206e-05,3.988e-05,6.606e-04,0.053,0.004,0.003,0.254,0.014,1.031e-04,2.905e-04,1.092e-05,0.014,0.037,0.014,0.003,0.022,4.827e-04,-3.058,5.147e-05,0.030,1.411e-04
EX30,0.002,3.781e-05,0.002,9.800e-04,0.054,0.013,1.107e-05,0.034,0.009,0.003,0.014,0.017,0.016,0.002,0.002,6.189e-05,3.585e-04,9.787e-05,2.970e-04,0.002,0.002,6.468e-04,1.000e-04,0.015,5.001e-04,...,0.007,0.014,2.663e-04,0.046,2.206e-05,3.988e-05,6.606e-04,0.053,0.004,0.003,0.254,0.014,1.031e-04,2.905e-04,1.092e-05,0.014,0.037,0.014,0.003,0.022,4.827e-04,0.008,-2.621e+00,0.030,1.411e-04


In [150]:
product_data['costs'] = GH_results.compute_costs()
product_data['profit_per_car'] = product_data['prices'] - product_data['costs']
product_data['markups'] = product_data['profit_per_car'] / product_data['costs']
product_data['profits'] = GH_results.compute_profits()
product_data[['prices', 'costs', 'profit_per_car', 'markups', 'profits']].describe()

,prices,costs,profit_per_car,markups,profits
count,334.000,334.000,334.000,334.000,3.340e+02
mean,46.911,32.757,14.154,1.020,3.076e-02
std,29.975,29.929,0.210,3.447,8.670e-02
min,12.486,-1.580,14.049,-16.345,6.107e-05
25%,28.398,14.333,14.059,0.352,1.727e-03
50%,35.904,21.725,14.082,0.642,1.012e-02
75%,52.962,38.628,14.188,0.975,3.047e-02
max,194.052,179.840,15.779,54.808,1.391e+00


In [151]:
product_data.loc[product_data['market_ids'] == 2023]

,ID,market_ids,shares,firm_ids,product_ids,Range,prices,HP,Chargetime,Type,Segment,Country,Sales,Intercept,demand_instruments0,demand_instruments1,demand_instruments2,China,outside_share,logit_delta,costs,profit_per_car,markups,profits
2,1,2023,8.690e-04,Aiways,U5,40.0,26.452,20.1,34,SUV,C,CN,177,1.0,27923,15010,2099,1,0.696,-6.686,12.391,14.061,1.135,1.222e-02
3,2,2023,1.473e-05,Aiways,U6,40.5,36.064,21.4,34,SUV,C,CN,3,1.0,27468,14562,2099,1,0.696,-10.764,22.002,14.061,0.639,2.071e-04
8,3,2023,3.338e-04,Audi,e-tron,37.5,67.304,40.2,17,SUV,F,DE,68,1.0,31201,19786,3157,0,0.696,-7.643,53.053,14.251,0.269,4.758e-03
11,4,2023,1.129e-04,Audi,e-tron GT,47.2,121.934,52.2,17,Sedan,F,DE,23,1.0,21894,21035,3157,0,0.696,-8.727,107.683,14.251,0.132,1.609e-03
14,5,2023,1.153e-02,Audi,Q4 e-tron,49.6,65.855,28.1,28,SUV,C,DE,2349,1.0,23156,17256,1813,0,0.696,-4.100,51.604,14.251,0.276,1.643e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,187,2023,3.878e-04,Volkswagen,up!,25.6,17.485,8.1,48,Hatchback,A,DE,79,1.0,37374,22690,2733,0,0.696,-7.493,2.997,14.488,4.833,5.619e-03
328,188,2023,2.538e-03,Volvo,C40,46.6,43.066,40.2,28,SUV,C,SE,517,1.0,22124,19786,1813,0,0.696,-5.614,28.844,14.223,0.493,3.610e-02
329,189,2023,1.964e-05,Volvo,EX30,47.5,36.825,26.8,28,SUV,B,SE,4,1.0,21894,17776,1813,0,0.696,-10.476,22.602,14.223,0.629,2.793e-04
332,190,2023,9.647e-03,Volvo,XC40,45.7,43.927,40.2,28,SUV,C,SE,1965,1.0,22067,19786,1813,0,0.696,-4.279,29.704,14.223,0.479,1.372e-01


In [152]:
statsmodels_ols = smf.ols('np.log(costs) ~ 1 + Range + HP + Chargetime', product_data)
statsmodels_results = statsmodels_ols.fit(cov_type='HC0')
statsmodels_results.summary2()

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
=================================================================
Model:              OLS              Adj. R-squared:     0.612   
Dependent Variable: np.log(costs)    AIC:                561.6596
Date:               2024-06-03 23:33 BIC:                576.8802
No. Observations:   332              Log-Likelihood:     -276.83 
Df Model:           3                F-statistic:        140.4   
Df Residuals:       328              Prob (F-statistic): 1.57e-58
R-squared:          0.615            Scale:              0.31408 
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept      2.3072    0.1539  14.9886  0.0000   2.0055   2.6089
Range          0.0184    0.0033   5.5302  0.0000   0.0119   0.0249
HP             0.0284    0.0020  13.9519  0.0000   0.0244   0.0324
Chargetime    -0.0182    0.0032  -5.5982  0.0000  -0.0245  -0.0118
-----------------------------------------------------------------
Omnibus:             88.949       Durbin-Watson:          0.777  
Prob(Omnibus):       0.000        Jarque-Bera (JB):       471.764
Skew:                -0.996       Prob(JB):               0.000  
Kurtosis:            8.489        Condition No.:          333    
=================================================================

"""

In [153]:
eq_prices = GH_results.compute_prices()
prices = pd.DataFrame(eq_prices, index = product_data['product_ids'])

In [154]:
prices.loc['Model Y']

,0
product_ids,
Model Y,50.324
Model Y,52.115
Model Y,40.555


In [155]:
product_data[product_data['product_ids'] == 'Model Y']['profits']

283    0.098
284    0.183
285    1.391
Name: profits, dtype: float64

# Counterfactual 1
### Add Manufacturer and year fixed effects

In [156]:
cf1 = product_data.copy()

In [157]:
fe_problem = pyblp.Problem(pyblp.Formulation('0 + prices + Range + HP + Chargetime', absorb='C(market_ids) + C(firm_ids)'), cf1)
fe_results = fe_problem.solve(method='1s')

In [158]:
cf1['costs'] = fe_results.compute_costs()
cf1['profit_per_car'] = cf1['prices'] - cf1['costs']
cf1['markups'] = cf1['profit_per_car'] / cf1['costs']
cf1['profits'] = fe_results.compute_profits()
cf1[['prices', 'costs', 'profit_per_car', 'markups', 'profits']].describe()

,prices,costs,profit_per_car,markups,profits
count,334.000,334.000,334.000,334.000,3.340e+02
mean,46.911,38.280,8.632,0.356,1.876e-02
std,29.975,29.946,0.128,0.264,5.287e-02
min,12.486,3.908,8.567,0.047,3.724e-05
25%,28.398,19.821,8.573,0.198,1.053e-03
50%,35.904,27.212,8.587,0.315,6.174e-03
75%,52.962,44.221,8.652,0.433,1.858e-02
max,194.052,185.386,9.622,2.195,8.481e-01


In [159]:
cf1[cf1['market_ids'] == 2023]

,ID,market_ids,shares,firm_ids,product_ids,Range,prices,HP,Chargetime,Type,Segment,Country,Sales,Intercept,demand_instruments0,demand_instruments1,demand_instruments2,China,outside_share,logit_delta,costs,profit_per_car,markups,profits
2,1,2023,8.690e-04,Aiways,U5,40.0,26.452,20.1,34,SUV,C,CN,177,1.0,27923,15010,2099,1,0.696,-6.686,17.878,8.575,0.480,7.451e-03
3,2,2023,1.473e-05,Aiways,U6,40.5,36.064,21.4,34,SUV,C,CN,3,1.0,27468,14562,2099,1,0.696,-10.764,27.489,8.575,0.312,1.263e-04
8,3,2023,3.338e-04,Audi,e-tron,37.5,67.304,40.2,17,SUV,F,DE,68,1.0,31201,19786,3157,0,0.696,-7.643,58.613,8.691,0.148,2.901e-03
11,4,2023,1.129e-04,Audi,e-tron GT,47.2,121.934,52.2,17,Sedan,F,DE,23,1.0,21894,21035,3157,0,0.696,-8.727,113.244,8.691,0.077,9.813e-04
14,5,2023,1.153e-02,Audi,Q4 e-tron,49.6,65.855,28.1,28,SUV,C,DE,2349,1.0,23156,17256,1813,0,0.696,-4.100,57.165,8.691,0.152,1.002e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,187,2023,3.878e-04,Volkswagen,up!,25.6,17.485,8.1,48,Hatchback,A,DE,79,1.0,37374,22690,2733,0,0.696,-7.493,8.650,8.835,1.021,3.427e-03
328,188,2023,2.538e-03,Volvo,C40,46.6,43.066,40.2,28,SUV,C,SE,517,1.0,22124,19786,1813,0,0.696,-5.614,34.393,8.673,0.252,2.201e-02
329,189,2023,1.964e-05,Volvo,EX30,47.5,36.825,26.8,28,SUV,B,SE,4,1.0,21894,17776,1813,0,0.696,-10.476,28.151,8.673,0.308,1.703e-04
332,190,2023,9.647e-03,Volvo,XC40,45.7,43.927,40.2,28,SUV,C,SE,1965,1.0,22067,19786,1813,0,0.696,-4.279,35.254,8.673,0.246,8.367e-02


# Counterfactual 2
### China decreases demand price by 20%
#### keep cf1

In [166]:
cf2_year = 2023
cf2 = cf1.loc[cf1['market_ids'] == cf2_year]
cf2

,ID,market_ids,shares,firm_ids,product_ids,Range,prices,HP,Chargetime,Type,Segment,Country,Sales,Intercept,demand_instruments0,demand_instruments1,demand_instruments2,China,outside_share,logit_delta,costs,profit_per_car,markups,profits
2,1,2023,8.690e-04,Aiways,U5,40.0,26.452,20.1,34,SUV,C,CN,177,1.0,27923,15010,2099,1,0.696,-6.686,17.878,8.575,0.480,7.451e-03
3,2,2023,1.473e-05,Aiways,U6,40.5,36.064,21.4,34,SUV,C,CN,3,1.0,27468,14562,2099,1,0.696,-10.764,27.489,8.575,0.312,1.263e-04
8,3,2023,3.338e-04,Audi,e-tron,37.5,67.304,40.2,17,SUV,F,DE,68,1.0,31201,19786,3157,0,0.696,-7.643,58.613,8.691,0.148,2.901e-03
11,4,2023,1.129e-04,Audi,e-tron GT,47.2,121.934,52.2,17,Sedan,F,DE,23,1.0,21894,21035,3157,0,0.696,-8.727,113.244,8.691,0.077,9.813e-04
14,5,2023,1.153e-02,Audi,Q4 e-tron,49.6,65.855,28.1,28,SUV,C,DE,2349,1.0,23156,17256,1813,0,0.696,-4.100,57.165,8.691,0.152,1.002e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,187,2023,3.878e-04,Volkswagen,up!,25.6,17.485,8.1,48,Hatchback,A,DE,79,1.0,37374,22690,2733,0,0.696,-7.493,8.650,8.835,1.021,3.427e-03
328,188,2023,2.538e-03,Volvo,C40,46.6,43.066,40.2,28,SUV,C,SE,517,1.0,22124,19786,1813,0,0.696,-5.614,34.393,8.673,0.252,2.201e-02
329,189,2023,1.964e-05,Volvo,EX30,47.5,36.825,26.8,28,SUV,B,SE,4,1.0,21894,17776,1813,0,0.696,-10.476,28.151,8.673,0.308,1.703e-04
332,190,2023,9.647e-03,Volvo,XC40,45.7,43.927,40.2,28,SUV,C,SE,1965,1.0,22067,19786,1813,0,0.696,-4.279,35.254,8.673,0.246,8.367e-02


In [168]:
cf2['new_prices'] = cf2['prices']
cf2.loc[cf2['China'] == 1, 'new_prices'] = 2
#cf2['new_shares'] = fe_results.compute_shares(market_id=cf2, prices=cf2['new_prices'])
#cf2['iv_change'] = 100 * (cf2['new_shares'] - cf2['shares']) / cf2['shares']
cf2

/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_94680/802951142.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf2['new_prices'] = cf2['prices']


,ID,market_ids,shares,firm_ids,product_ids,Range,prices,HP,Chargetime,Type,Segment,Country,Sales,Intercept,demand_instruments0,demand_instruments1,demand_instruments2,China,outside_share,logit_delta,costs,profit_per_car,markups,profits,new_prices
2,1,2023,8.690e-04,Aiways,U5,40.0,26.452,20.1,34,SUV,C,CN,177,1.0,27923,15010,2099,1,0.696,-6.686,17.878,8.575,0.480,7.451e-03,2.000
3,2,2023,1.473e-05,Aiways,U6,40.5,36.064,21.4,34,SUV,C,CN,3,1.0,27468,14562,2099,1,0.696,-10.764,27.489,8.575,0.312,1.263e-04,2.000
8,3,2023,3.338e-04,Audi,e-tron,37.5,67.304,40.2,17,SUV,F,DE,68,1.0,31201,19786,3157,0,0.696,-7.643,58.613,8.691,0.148,2.901e-03,67.304
11,4,2023,1.129e-04,Audi,e-tron GT,47.2,121.934,52.2,17,Sedan,F,DE,23,1.0,21894,21035,3157,0,0.696,-8.727,113.244,8.691,0.077,9.813e-04,121.934
14,5,2023,1.153e-02,Audi,Q4 e-tron,49.6,65.855,28.1,28,SUV,C,DE,2349,1.0,23156,17256,1813,0,0.696,-4.100,57.165,8.691,0.152,1.002e-01,65.855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,187,2023,3.878e-04,Volkswagen,up!,25.6,17.485,8.1,48,Hatchback,A,DE,79,1.0,37374,22690,2733,0,0.696,-7.493,8.650,8.835,1.021,3.427e-03,17.485
328,188,2023,2.538e-03,Volvo,C40,46.6,43.066,40.2,28,SUV,C,SE,517,1.0,22124,19786,1813,0,0.696,-5.614,34.393,8.673,0.252,2.201e-02,43.066
329,189,2023,1.964e-05,Volvo,EX30,47.5,36.825,26.8,28,SUV,B,SE,4,1.0,21894,17776,1813,0,0.696,-10.476,28.151,8.673,0.308,1.703e-04,36.825
332,190,2023,9.647e-03,Volvo,XC40,45.7,43.927,40.2,28,SUV,C,SE,1965,1.0,22067,19786,1813,0,0.696,-4.279,35.254,8.673,0.246,8.367e-02,43.927
